In [3]:
from dataloaders import BertDataloader, BertDataset
from templates import set_template
from options import args

/home/aiden/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
set_template(args)

In [5]:

from tmpdatasets import dataset_factory
dataset = dataset_factory(args)
dataloader = BertDataloader(args, dataset)
train_loader, val_loader, test_loader = dataloader.get_pytorch_dataloaders()


Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.


In [6]:
u2seq = dataloader.train
u2answer = dataloader.val
max_len = dataloader.max_len
mask_prob = dataloader.mask_prob
CLOZE_MASK_TOKEN = dataloader.CLOZE_MASK_TOKEN
item_count = dataloader.item_count
rng = dataloader.rng


In [11]:
print(item_count)
print(max_len)
print(mask_prob)
print(CLOZE_MASK_TOKEN)

3706
100
0.15


In [8]:
trainset = BertDataset(
    mask_token = CLOZE_MASK_TOKEN,
    u2seq=u2seq,
    max_len = max_len,
    mask_prob = mask_prob,    
    num_items = item_count,
    rng = rng
)

In [7]:
valset = BertDataset(
    eval=True,
    u2seq=u2seq,
    u2answer=u2answer,
    max_len = max_len,
    mask_token = CLOZE_MASK_TOKEN,
    negative_samples = dataloader.test_negative_samples
)

In [8]:
from model import BERTModel

mymodel = BERTModel(
    hidden_size=256,
    num_items=train_loader.dataset.num_items,     # item 的數量
    n_layers=2,
    dropout=0,
    heads=8,
    max_len=100,
)

In [9]:
trainset

In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    trainset, 
    batch_size=dataloader.args.train_batch_size,
    shuffle=True,
    pin_memory=True
)

val_loader = DataLoader(
    valset,
    batch_size=dataloader.args.val_batch_size,
    shuffle=False,
    pin_memory=True
)

In [11]:
for i in train_loader:
    print(i)
    break

[tensor([[  31,  466, 2374,  ..., 2765, 3707, 3512],
        [2755, 3620, 1044,  ..., 3132, 3707,  158],
        [   0,    0,    0,  ..., 3707, 1384, 3707],
        ...,
        [   0,    0,    0,  ..., 3707,  428, 3707],
        [   0,    0,    0,  ..., 1833, 3707, 3707],
        [   0,    0,    0,  ...,  159, 1271, 2778]]), tensor([[   0,    0,    0,  ...,    0, 3627, 3512],
        [   0, 1585,    0,  ...,    0, 1210,    0],
        [   0,    0,    0,  ...,   46,    0, 3266],
        ...,
        [   0,    0,    0,  ..., 1022, 3112,  155],
        [   0,    0,    0,  ...,    0, 1272, 2174],
        [   0,    0,    0,  ...,    0,    0,    0]]), tensor([], size=(128, 0))]


In [12]:
import pytorch_lightning as pl
trainer = pl.Trainer(limit_train_batches=100, max_epochs=10, gpus=1)
trainer.fit(mymodel, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | bert | BERT   | 2.6 M 
1 | out  | Linear | 952 K 
--------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
14.028    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/aiden/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/aiden/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/aiden/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (48) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 9: 100%|██████████| 96/96 [00:04<00:00, 21.01it/s, loss=5.63, v_num=1]
